In [2]:
import pandas as pd
from rank_bm25 import BM25Okapi
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel

df=pd.read_csv('booksdatasetmodified.csv')
df=df[['text','category']]


# Number of rows to choose from each category
rows_per_category = 100

# Select 500 rows from each category
dfs = df.groupby('category').head(rows_per_category).reset_index(drop=True)

df1=dfs[['text','category']]
df2 = pd.read_csv('usertraininput.csv')

df2['query'] = df2['input'].str.split('article:', expand=True)[1]


# Retrieval

In [3]:
def retrieval_model_b(query,user_profile,top_k):
    data_list = eval(user_profile)
    df_ = pd.DataFrame(data_list)
    df_ = df_.drop(columns=['id'])
    df_ = df_.drop(columns=['title'])
    combined_df=pd.DataFrame()
    combined_df = pd.concat([df_, df1], ignore_index=True)
    ls=[]
    for i in combined_df['text']:
        ls.append(i)
    
    tokenized_corpus = [doc.split() if isinstance(doc, str) else [] for doc in ls]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)
    doc_scores = list(enumerate(zip(scores,  list(combined_df[['text', 'category']].to_dict('records')))))
    sorted_docs = sorted(doc_scores, key=lambda x: x[1][0], reverse=True)[:top_k]
    formatted_entries = [
    f'The category for the article: "{entry[1][1]["text"]}" is "{entry[1][1]["category"]}"'
    for entry in sorted_docs]
    result_string = '\n'.join(formatted_entries)
    return result_string
    

In [4]:
bm25_results=[]
for i in range(len(df2)):
    bm25_results.append(retrieval_model_b(df2['query'][i],df2['profile'][i],5))

In [5]:
training_bm25=[]
for i in range(len(bm25_results)):
    training_bm25.append(bm25_results[i]+' The above are similar articles for the query asked further.'+df2['input'][i])
    

# Flan-T5 base finetuned

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [10]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
dfo = pd.read_csv('usertrainoutput.csv')
trainoutput=dfo['output'].to_list()
ytrain=trainoutput[:int(len(trainoutput)*0.8)]
ytest=trainoutput[int(len(trainoutput)*0.8):]

In [19]:
def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [doc for doc in examples["nbestanswers"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["category"], 
                      max_length=512,         
                      truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    print(model_inputs["labels"])
    return model_inputs

In [15]:
import pandas as pd

data = {
    'id': dfo['id'],
    'input': df2['input'],
    'nbestanswers': training_bm25,
    'category': dfo['output']
}

dfn = pd.DataFrame(data)


In [17]:
from datasets import Dataset, DatasetDict
converted_dataset = Dataset.from_pandas(dfn)
books_categorization = DatasetDict({
    'train': converted_dataset
})

In [18]:
books_categorization = books_categorization["train"].train_test_split(test_size=0.2)

In [20]:
books_categorization.map(preprocess_function, batched=True)

Map:   0%|          | 0/4731 [00:00<?, ? examples/s]

[[6525, 1], [6525, 1], [5447, 1], [1695, 840, 1], [4527, 1], [1543, 3, 184, 5138, 1], [268, 1], [869, 3, 184, 2790, 1], [6525, 1], [4527, 1], [6525, 1], [4527, 1], [2100, 1], [1695, 840, 1], [6525, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [1695, 840, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [268, 1], [6525, 1], [869, 3, 184, 2790, 1], [4527, 1], [4527, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [2100, 1], [1695, 840, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [268, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [2100, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [887, 1], [6525, 1], [6525, 1], [5

[[6525, 1], [6525, 1], [6525, 1], [887, 1], [6525, 1], [2056, 3, 184, 748, 1], [887, 1], [5562, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [5562, 1], [887, 1], [6525, 1], [268, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [2056, 3, 184, 748, 1], [2056, 3, 184, 748, 1], [4527, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [1695, 840, 1], [6525, 1], [2100, 1], [4527, 1], [6525, 1], [6525, 1], [1695, 840, 1], [2056, 3, 184, 748, 1], [6525, 1], [6525, 1], [1362, 1], [1543, 3, 184, 5138, 1], [1111, 1], [4527, 1], [6525, 1], [1111, 1], [4527, 1], [4527, 1], [4527, 1], [542, 3, 184, 3281, 1], [6525, 1], [6525, 1], [6525, 1], [1073, 1], [2100, 1], [1111, 1], [6525, 1], [268, 1], [6525, 1], [1695, 840, 1], [6525, 1], [4527, 1], [1111, 1], [268, 1], [2056, 3, 184, 748, 1], [1111, 1], [887, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [887, 1], [6525, 1], [2100, 1], [4527, 1], [4527, 1], [4527, 1], [65

[[2056, 3, 184, 748, 1], [6525, 1], [869, 3, 184, 2790, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [1111, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [1111, 1], [542, 3, 184, 3281, 1], [2100, 1], [6525, 1], [6525, 1], [5562, 1], [887, 1], [6525, 1], [1695, 840, 1], [4527, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [5447, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [1543, 3, 184, 5138, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [4527, 1], [6525, 1], [268, 1], [6525, 1], [268, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [2100, 1], [6525, 1], [1695, 840, 1], [268, 1], [542, 3, 184, 3281, 1], [1695, 840, 1], [6525, 1], [1695, 840, 1], [1111, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1695, 840, 1], [1111, 1], [869, 3, 184, 2790, 1], [6525, 1], [1695, 840, 1], [4527, 1], [4527, 1], [4527, 1], [6525, 1], [6525, 1], [

[[268, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [887, 1], [4527, 1], [6525, 1], [5447, 1], [1695, 840, 1], [2100, 1], [2100, 1], [1543, 3, 184, 5138, 1], [6525, 1], [4527, 1], [268, 1], [4527, 1], [6525, 1], [1111, 1], [1111, 1], [6525, 1], [2100, 1], [2100, 1], [1111, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [1543, 3, 184, 5138, 1], [1073, 1], [268, 1], [4527, 1], [6525, 1], [1695, 840, 1], [6525, 1], [887, 1], [6525, 1], [268, 1], [4527, 1], [6525, 1], [1073, 1], [6525, 1], [6525, 1], [6525, 1], [1111, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [6525, 1], [2100, 1], [6525, 1], [6525, 1], [6525, 1], [1111, 1], [6525, 1], [6525, 1], [1543, 3, 184, 5138, 1], [6525, 1], [6525, 1], [6525, 1], [5562, 1], [6525, 1], [6525, 1], [869, 3, 184, 2790, 1], [869, 3, 184, 2790, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1695, 840, 1], [6525, 1], [1111, 1], [6525, 

[[6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [869, 3, 184, 2790, 1], [6525, 1], [2100, 1], [542, 3, 184, 3281, 1], [1111, 1], [6525, 1], [6525, 1], [4527, 1], [1362, 1], [6525, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [6525, 1], [6525, 1], [887, 1], [6525, 1], [6525, 1], [6525, 1], [2100, 1], [6525, 1], [1695, 840, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [887, 1], [4527, 1], [6525, 1], [4527, 1], [887, 1], [6525, 1], [268, 1], [6525, 1], [4527, 1], [268, 1], [6525, 1], [6525, 1], [1543, 3, 184, 5138, 1], [542, 3, 184, 3281, 1], [6525, 1], [2056, 3, 184, 748, 1], [5447, 1], [542, 3, 184, 3281, 1], [542, 3, 184, 3281, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [1362, 1], [6525, 1], [4527, 1], [1695, 840, 1], [268, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [887, 1], [4527, 1], [6525, 1], [869, 3, 184, 2790, 1], [869, 3, 184, 27

Map:   0%|          | 0/1183 [00:00<?, ? examples/s]

[[5447, 1], [1111, 1], [6525, 1], [1073, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1111, 1], [1543, 3, 184, 5138, 1], [4527, 1], [4527, 1], [6525, 1], [4527, 1], [6525, 1], [887, 1], [6525, 1], [4527, 1], [1362, 1], [2056, 3, 184, 748, 1], [268, 1], [268, 1], [4527, 1], [2100, 1], [6525, 1], [1695, 840, 1], [6525, 1], [4527, 1], [1695, 840, 1], [2056, 3, 184, 748, 1], [1543, 3, 184, 5138, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1695, 840, 1], [5447, 1], [1111, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [5447, 1], [4527, 1], [1362, 1], [4527, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [869, 3, 184, 2790, 1], [5447, 1], [1111, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [2100, 1], [542, 3, 184, 3281, 1], [1695, 840, 1], [6525, 1], [4527, 1], [268, 1], [6525, 1], [542, 3, 184, 3281, 1], [4527, 1], [6525, 1], [6525, 1], [2100, 1], [4527, 1], [4527, 1

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'nbestanswers', 'category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4731
    })
    test: Dataset({
        features: ['id', 'input', 'nbestanswers', 'category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1183
    })
})

In [21]:
# Map the preprocessing function across our dataset
tokenized_dataset = books_categorization.map(preprocess_function, batched=True)

Map:   0%|          | 0/4731 [00:00<?, ? examples/s]

[[6525, 1], [6525, 1], [5447, 1], [1695, 840, 1], [4527, 1], [1543, 3, 184, 5138, 1], [268, 1], [869, 3, 184, 2790, 1], [6525, 1], [4527, 1], [6525, 1], [4527, 1], [2100, 1], [1695, 840, 1], [6525, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [1695, 840, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [268, 1], [6525, 1], [869, 3, 184, 2790, 1], [4527, 1], [4527, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [2100, 1], [1695, 840, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [268, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [2100, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [887, 1], [6525, 1], [6525, 1], [5

[[6525, 1], [6525, 1], [6525, 1], [887, 1], [6525, 1], [2056, 3, 184, 748, 1], [887, 1], [5562, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [5562, 1], [887, 1], [6525, 1], [268, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [2056, 3, 184, 748, 1], [2056, 3, 184, 748, 1], [4527, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [1695, 840, 1], [6525, 1], [2100, 1], [4527, 1], [6525, 1], [6525, 1], [1695, 840, 1], [2056, 3, 184, 748, 1], [6525, 1], [6525, 1], [1362, 1], [1543, 3, 184, 5138, 1], [1111, 1], [4527, 1], [6525, 1], [1111, 1], [4527, 1], [4527, 1], [4527, 1], [542, 3, 184, 3281, 1], [6525, 1], [6525, 1], [6525, 1], [1073, 1], [2100, 1], [1111, 1], [6525, 1], [268, 1], [6525, 1], [1695, 840, 1], [6525, 1], [4527, 1], [1111, 1], [268, 1], [2056, 3, 184, 748, 1], [1111, 1], [887, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [887, 1], [6525, 1], [2100, 1], [4527, 1], [4527, 1], [4527, 1], [65

[[2056, 3, 184, 748, 1], [6525, 1], [869, 3, 184, 2790, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [1111, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [1111, 1], [542, 3, 184, 3281, 1], [2100, 1], [6525, 1], [6525, 1], [5562, 1], [887, 1], [6525, 1], [1695, 840, 1], [4527, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [5447, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [1543, 3, 184, 5138, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [4527, 1], [6525, 1], [268, 1], [6525, 1], [268, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [2100, 1], [6525, 1], [1695, 840, 1], [268, 1], [542, 3, 184, 3281, 1], [1695, 840, 1], [6525, 1], [1695, 840, 1], [1111, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1695, 840, 1], [1111, 1], [869, 3, 184, 2790, 1], [6525, 1], [1695, 840, 1], [4527, 1], [4527, 1], [4527, 1], [6525, 1], [6525, 1], [

[[268, 1], [6525, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [887, 1], [4527, 1], [6525, 1], [5447, 1], [1695, 840, 1], [2100, 1], [2100, 1], [1543, 3, 184, 5138, 1], [6525, 1], [4527, 1], [268, 1], [4527, 1], [6525, 1], [1111, 1], [1111, 1], [6525, 1], [2100, 1], [2100, 1], [1111, 1], [6525, 1], [6525, 1], [6525, 1], [5447, 1], [1543, 3, 184, 5138, 1], [1073, 1], [268, 1], [4527, 1], [6525, 1], [1695, 840, 1], [6525, 1], [887, 1], [6525, 1], [268, 1], [4527, 1], [6525, 1], [1073, 1], [6525, 1], [6525, 1], [6525, 1], [1111, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [2056, 3, 184, 748, 1], [6525, 1], [2100, 1], [6525, 1], [6525, 1], [6525, 1], [1111, 1], [6525, 1], [6525, 1], [1543, 3, 184, 5138, 1], [6525, 1], [6525, 1], [6525, 1], [5562, 1], [6525, 1], [6525, 1], [869, 3, 184, 2790, 1], [869, 3, 184, 2790, 1], [6525, 1], [5447, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1695, 840, 1], [6525, 1], [1111, 1], [6525, 

[[6525, 1], [6525, 1], [6525, 1], [4527, 1], [6525, 1], [869, 3, 184, 2790, 1], [6525, 1], [2100, 1], [542, 3, 184, 3281, 1], [1111, 1], [6525, 1], [6525, 1], [4527, 1], [1362, 1], [6525, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [6525, 1], [6525, 1], [887, 1], [6525, 1], [6525, 1], [6525, 1], [2100, 1], [6525, 1], [1695, 840, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [887, 1], [4527, 1], [6525, 1], [4527, 1], [887, 1], [6525, 1], [268, 1], [6525, 1], [4527, 1], [268, 1], [6525, 1], [6525, 1], [1543, 3, 184, 5138, 1], [542, 3, 184, 3281, 1], [6525, 1], [2056, 3, 184, 748, 1], [5447, 1], [542, 3, 184, 3281, 1], [542, 3, 184, 3281, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [542, 3, 184, 3281, 1], [6525, 1], [1362, 1], [6525, 1], [4527, 1], [1695, 840, 1], [268, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [4527, 1], [887, 1], [4527, 1], [6525, 1], [869, 3, 184, 2790, 1], [869, 3, 184, 27

Map:   0%|          | 0/1183 [00:00<?, ? examples/s]

[[5447, 1], [1111, 1], [6525, 1], [1073, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1111, 1], [1543, 3, 184, 5138, 1], [4527, 1], [4527, 1], [6525, 1], [4527, 1], [6525, 1], [887, 1], [6525, 1], [4527, 1], [1362, 1], [2056, 3, 184, 748, 1], [268, 1], [268, 1], [4527, 1], [2100, 1], [6525, 1], [1695, 840, 1], [6525, 1], [4527, 1], [1695, 840, 1], [2056, 3, 184, 748, 1], [1543, 3, 184, 5138, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [1695, 840, 1], [5447, 1], [1111, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [4527, 1], [5447, 1], [4527, 1], [1362, 1], [4527, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [6525, 1], [869, 3, 184, 2790, 1], [5447, 1], [1111, 1], [869, 3, 184, 2790, 1], [4527, 1], [6525, 1], [6525, 1], [6525, 1], [2100, 1], [542, 3, 184, 3281, 1], [1695, 840, 1], [6525, 1], [4527, 1], [268, 1], [6525, 1], [542, 3, 184, 3281, 1], [4527, 1], [6525, 1], [6525, 1], [2100, 1], [4527, 1], [4527, 1

In [22]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("f1","accuracy")

In [23]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results_books",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [24]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.553200,0.543440
2,0.434200,0.552582
3,0.379000,0.587731


TrainOutput(global_step=1776, training_loss=0.4424168011089703, metrics={'train_runtime': 1788.9362, 'train_samples_per_second': 7.934, 'train_steps_per_second': 0.993, 'total_flos': 2429690275823616.0, 'train_loss': 0.4424168011089703, 'epoch': 3.0})

# TESTING

In [26]:
last_checkpoint = "./results_books/checkpoint-1500"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
dfxt=pd.read_csv('uservalinput.csv')
dfyt=pd.read_csv('uservaloutput.csv')

dfxt['query'] = dfxt['input'].str.split('article:', expand=True)[1]

bm25_results_test=[]
for i in range(len(dfxt)):
    bm25_results_test.append(retrieval_model_b(dfxt['query'][i],dfxt['profile'][i],5))
    
test_bm25=[]
for i in range(len(bm25_results_test)):
    test_bm25.append(bm25_results_test[i]+' The above are similar articles for the query asked further.'+dfxt['input'][i])
    
data = {
    'id': dfxt['id'],
    'input': dfxt['input'],
    'nbestanswers': test_bm25,
    'category': dfyt['output']
}

dfnt = pd.DataFrame(data)

prompts=dfnt['nbestanswers'].to_list()

y_true=dfnt['category'].to_list()

In [29]:
preds=[]
for q in prompts:
    inputs = tokenizer(q, return_tensors="pt")
    outputs = finetuned_model.generate(**inputs)
    answer = tokenizer.decode(outputs[0])
    answer=answer.split('<pad>')
    answer=answer[1].split('</s>')
    answer=answer[0].strip()
    preds.append(answer)
    

Token indices sequence length is longer than the specified maximum sequence length for this model (1375 > 512). Running this sequence through the model will result in indexing errors
/Users/rithikaflorianjohnson/anaconda3/envs/ml/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [30]:
from sklearn.metrics import accuracy_score, f1_score


accuracy = accuracy_score(y_true, preds)

# Calculate F1 Score
f1 = f1_score(y_true, preds,average='weighted')

# Print the results
print("Accuracy:", accuracy)
print("F1 Score:", f1)

Accuracy: 0.7110266159695817
F1 Score: 0.6672131719475899
